### Prompt Templates
* Introduce programming concepts into your conversations with the LLM.
* A Prompt Template is a predefined structure that helps generate inputs (prompts) for language models in a reusable, consistent, and flexible way. It serves as a blueprint for creating dynamic prompts by incorporating variables and specific instructions.

### Intro
* Input: the prompt we send to the LLM.
* Output: the response from the LLM.
* We can switch LLMs and use several different LLMs. 

### Why Are Prompt Templates Necessary?
- **Consistency:** Ensures that prompts follow a specific structure, reducing errors and ambiguities.
- **Reusability:** Templates can be reused across multiple tasks or workflows.
- **Dynamic Input Handling:** Supports variable substitution to customize prompts for different contexts.
- **Improved Performance:** Provides structured and optimized inputs for language models, improving their effectiveness.

### categorization of Prompt Types:

- **Template Types** are categorized based on the type or role of the message (system, human, AI) and how the prompt is structured.
- **Shot Types** are categorized based on how much context or how many examples (messages) you provide to help the model understand the task.


#### A. Template Types (Categorization Based on Type) 
* These are categories based on the structure or format of the prompt. They focus on what type of message or content is being used in the conversation. For example:

##### 1. PromptTemplate
- **Description:**
  - A general-purpose template for creating prompts.
  - Supports variables that can be substituted with dynamic values.
- **Use Case:**
  - Works best for single-shot or basic scenarios, such as question-answering or text generation.

---

##### 2. ChatPromptTemplate
- **Description:**
  - A specialized template for managing multi-turn conversations.
  - Combines multiple message types (e.g., system, human, AI).
- **Use Case:**
  - Ideal for chat-based interactions with Large Language Models (LLMs) like ChatGPT.

---

##### 3. SystemMessagePromptTemplate
- **Description:**
  - Represents system-level instructions that guide the AI's behavior.
  - Defines the role, tone, or rules for the model during the interaction.
- **Use Case:**
  - Used at the beginning of a chat to set the AI's context.

---

##### 4. HumanMessagePromptTemplate
- **Description:**
  - Represents messages from the user in a chat.
  - Typically contains user queries or instructions.
- **Use Case:**
  - Models conversational input from a human.

---

##### 5. AIMessagePromptTemplate
- **Description:**
  - Represents messages generated by the AI in a conversation.
  - Models AI responses to user inputs.
- **Use Case:**
  - Helps create structured AI responses during prompt design.

---
---

- **library:** Following library has those prompts
     - from langchain_core.prompts import (</br>
                                            ChatPromptTemplate,</br>
                                            PromptTemplate,</br>
                                            SystemMessagePromptTemplate,</br>
                                            AIMessagePromptTemplate,</br>
                                            HumanMessagePromptTemplate,</br>
                                    )
    
     - **functions:** Following functions commonly used with thoses prompts , How to use will varies prompt to prompt case.
        - **from_messages()**   : Works with `ChatPromptTemplate`
        - **format_message()**   : Works with `ChatPromptTemplate`
        - **format_prompt()**    : Works with all Template Types
        - **from_template()**    : Works with all Template Types
        - **format()**           : Works with all Template Types
        - **invoke()**           : Works with all Template Types

In [1]:
## Following code will fetch the key-value pairs from the environment variables of the program
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # Althouh it will return True if the key-value pairs are fetched successfully, but we don't need to store it in any variable.
openai_api_key = os.environ["OPENAI_API_KEY"]
google_api_key=os.environ["GOOGLE_API_KEY"]
cerebras_api_key=os.environ["CEREBRAS_API_KEY"]

##### This following syntax is valid for PromptTemplate

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
# Load environment variables from a .env file.
load_dotenv()
# For no input, you remove the variables from the template string and make input_variables=[] (i.e., an empty list).
multiple_input_prompt = PromptTemplate(
    input_variables=["topic", "level"],
    template="Tell me a fact about {topic} for a student {level} level."
)
# Format the prompt, similar to f-string in Python.
# prompt=multiple_input_prompt.invoke({"topic":'Mars', "level":'8th Grade'})
# prompt=multiple_input_prompt.format(topic='Mars', level='8th Grade')
prompt=multiple_input_prompt.format_prompt(topic='Mars', level='8th Grade')
# prompt=multiple_input_prompt.format_message(topic='Mars', level='8th Grade')   # This will not work as no function format_message
# Initialize the Google Generative AI model (gemini-1.5-pro).
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
# Invoke the LLM and print the result based on the prompt.
response=llm.invoke(prompt)
print(response.content)

##### This syntax is valid for ChatPromptTemplate

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    ("human", "Tell me {joke_count} jokes."),
]
chatprompt_template = ChatPromptTemplate.from_messages(messages)
# prompt = chatprompt_template.invoke({"topic": "lawyers", "joke_count": 3})                                 
# prompt = chatprompt_template.format(topic="lawyers", joke_count=3)
# prompt = chatprompt_template.format_prompt(topic="lawyers", joke_count=3)
prompt = chatprompt_template.format_prompt(topic="lawyers", joke_count=3).to_messages()
# prompt = chatprompt_template.format_messages(topic="lawyers", joke_count=3)
print("\n----- Prompt with System and Human Messages (Tuple) -----\n")
print(prompt)
# Initialize the Google Generative AI model (gemini-1.5-pro)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
# Invoke the LLM and print the result based on the prompt
print(llm.invoke(prompt)) 

####  Some Pointers related to message that we supose to pass LLms.
##### Note:A scenario that does NOT work

- The attempt to use a dynamic human message with placeholders (HumanMessage(content="Tell me {joke_count} jokes.")) does not function as expected.

```python

messages = [
    ("system", "You are a comedian who tells jokes about {topic}."),
    HumanMessage(content="Tell me {joke_count} jokes."),
]
prompt_template = ChatPromptTemplate.from_messages(messages)
prompt = prompt_template.invoke({"topic": "lawyers", "joke_count": 3})
print("\n----- Prompt with System and Human Messages (Tuple) -----\n")
print(prompt)

##### Message Passing as a List of Tuples:

Here, we need to maintain the order of the messages: **system**, **human**, and **AI**. The **system** is optional. If the **system** is included, it will appear only once and must be placed first, as it is used to set the context of the chat.

The pair of **human** and **AI** can appear any number of times. In each pair, **human** must come first, followed by **AI**. At the end, **human** will be the last message.

- **Human** 
- **System** 
- **AI** 

##### Some More Example: Using .from_message() function


In [ ]:
from langchain_groq import ChatGroq  # llama-3.1-70b-versatile
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
messages=[
    ("human", "What is the capital of {country}?"),
    ("ai", "The capital of {country} is {capital}."),
    ("human", "Tell me above answer is correct."),
   
]
chatprompt_template = ChatPromptTemplate.from_messages(messages)
# prompt = chatprompt_template.invoke({"topic": "lawyers", "joke_count": 3})                                 
# prompt = chatprompt_template.format(topic="lawyers", joke_count=3)
# prompt = chatprompt_template.format_prompt(topic="lawyers", joke_count=3)
prompt = chatprompt_template.format_prompt(country="Canada",capital="Winnipeg").to_messages()
# prompt = prompt_template.format_messages(topic="lawyers", joke_count=3)
print("\n----- Prompt with System and Human Messages (Tuple) -----\n")
print(prompt)
# Initialize the Google Generative AI model (gemini-1.5-pro)
llm = ChatGroq(model="llama-3.1-70b-versatile")
# Invoke the LLM and print the result based on the prompt
print((llm.invoke(prompt)).content)  

In [ ]:
from langchain_cohere import ChatCohere
from dotenv import load_dotenv # Importing the load_dotenv function from the dotenv module
from langchain_core.prompts import (
                                            ChatPromptTemplate,
                                            PromptTemplate,
                                            SystemMessagePromptTemplate,
                                            AIMessagePromptTemplate,
                                            HumanMessagePromptTemplate,
                                    )


system_template="You are an AI recipe assistant that specializes in {dietary_preference} dishes that can be prepared in {cooking_time}."
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template="{recipe_request}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
request = chat_prompt.invoke({"cooking_time":"15 min", "dietary_preference":"Vegan", "recipe_request":"Quick Snack"})
# request = chat_prompt.format(cooking_time="15 min", dietary_preference="Vegan", recipe_request="Quick Snack")
request = chat_prompt.format_messages(cooking_time="15 min", dietary_preference="Vegan", recipe_request="Quick Snack")
# request = chat_prompt.format_prompt(cooking_time="15 min", dietary_preference="Vegan", recipe_request="Quick Snack").to_messages()                           
load_dotenv() # Loading environment variables from a .env file
chat = ChatCohere()
result = chat.invoke(request)
print(result.content)

##### This syntax is valid for all types of prompt. using .from_templates()

In [ ]:

from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
template = "Tell me a joke about {topic}."
prompt_template = PromptTemplate.from_template(template)
print("-----Prompt from Template-----")
prompt = prompt_template.invoke({"topic": "dogs"})

#<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<OR>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
                     
prompt = prompt_template.format(topic='dogs')
print(prompt)
# Initialize the Google Generative AI model (gemini-1.5-pro)
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro")
# Invoke the LLM and print the result based on the prompt
print(llm.invoke(prompt)) 

#### B. Shot Types (Categorization Based on Messages or Examples):
* These are categories based on the number of examples or context you provide to the model. This refers to how much information (or how many messages/examples) you pass to the model to help it understand the task. For Examples:

### Example of Shot Types:

- **Zero-shot**: No examples given, the model generates responses based only on instructions.
- **One-shot**: A single example is given to help the model understand the task.
- **Few-shot**: A few examples are given to the model.
- **Many-shot**: Many examples are provided for better context.

Shot types categorize prompts based on how much context or how many message examples are included to guide the model in generating a response.

# Few Shot Prompt Template

Let's go over an example where you want a historical conversation to show the LLM Chat Bot a few examples, known as "Few Shot Prompts". We essentially build a historical conversation *before* sending the message history to the chat bot. Be careful not too make the entire message too long, as you may hit context limits (but the latest models have quite large contexsts, e.g. GPT-4 has up to 32k tokens at this time).

In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage,HumanMessage,AIMessage
from langchain_core.prompts import PromptTemplate,ChatPromptTemplate,SystemMessagePromptTemplate,AIMessagePromptTemplate,HumanMessagePromptTemplate
chat = ChatOpenAI(temperature=0)


In [3]:
#### Creating Example Inputs and Outputs
template = "You are a helpful assistant that translates complex legal terms into plain and understandable terms."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)

legal_text = "The provisions herein shall be severable, and if any provision or portion thereof is deemed invalid, illegal, or unenforceable by a court of competent jurisdiction, the remaining provisions or portions thereof shall remain in full force and effect to the maximum extent permitted by law."
example_input_one = HumanMessagePromptTemplate.from_template(legal_text)

plain_text = "The rules in this agreement can be separated. If a court decides that one rule or part of it is not valid, illegal, or cannot be enforced, the other rules will still apply and be enforced as much as they can under the law."
example_output_one = AIMessagePromptTemplate.from_template(plain_text)

human_template = "{legal_text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [4]:
chat_prompt = ChatPromptTemplate.from_messages(
    [system_message_prompt, example_input_one, example_output_one, human_message_prompt]
)
some_example_text = "The grantor, being the fee simple owner of the real property herein described, conveys and warrants to the grantee, his heirs and assigns, all of the grantor's right, title, and interest in and to the said property, subject to all existing encumbrances, liens, and easements, as recorded in the official records of the county, and any applicable covenants, conditions, and restrictions affecting the property, in consideration of the sum of [purchase price] paid by the grantee."
request = chat_prompt.format_prompt(legal_text=some_example_text).to_messages()

result = chat.invoke(request)
print(result.content)

The person selling the property, who owns it completely, is transferring all their rights and ownership to the buyer and their successors. This includes any existing debts, claims, or rights that are officially recorded, as well as any rules or limitations that affect the property. This transfer is in exchange for the amount of money paid by the buyer.
